In [5]:
import torch
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

from pulser.devices import Chadoq2

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)


def qubo_register_coords(Q):
    """Compute coordinates for register."""
    bitstrings = [np.binary_repr(i, len(Q)) for i in range(len(Q) ** 2)]
    costs = []
    # this takes exponential time with the dimension of the QUBO
    for b in bitstrings:
        z = np.array(list(b), dtype=int)
        cost = z.T @ Q @ z
        costs.append(cost)
    zipped = zip(bitstrings, costs)
    sort_zipped = sorted(zipped, key=lambda x: x[1])

    def evaluate_mapping(new_coords, *args):
        """Cost function to minimize. Ideally, the pairwise
        distances are conserved"""
        Q, shape = args
        new_coords = np.reshape(new_coords, shape)
        new_Q = squareform(Chadoq2.interaction_coeff / pdist(new_coords) ** 6)
        return np.linalg.norm(new_Q - Q)

    shape = (len(Q), 2)
    costs = []
    np.random.seed(0)
    x0 = np.random.random(shape).flatten()
    res = minimize(
        evaluate_mapping,
        x0,
        args=(Q, shape),
        method="Nelder-Mead",
        tol=1e-6,
        options={"maxiter": 200000, "maxfev": None},
    )
    return [(x, y) for (x, y) in np.reshape(res.x, (len(Q), 2))]

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from qadence import chain
from qadence import QuantumModel, QuantumCircuit, Register
from qadence import RydbergDevice, AnalogRZ, AnalogRX

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
def cost_colouring(bitstring, Q):
    z = np.array(list(bitstring), dtype=int)
    cost = z.T @ Q @ z
    return cost

# Cost function.
def cost_fn(counter, Q):
    cost = sum(counter[key] * cost_colouring(key, Q) for key in counter)
    return cost / sum(counter.values())  # Divide by total samples


# Weights.
Q = np.array(
    [
        [-0.01074377,  0.04727714,  0.04727714,  0.04727714],
        [ 0.04727714, -0.00892812,  0.03218907,  0.04727714],
        [ 0.04727714,  0.03218907, -0.00865155,  0.04727714],
        [ 0.04727714,  0.04727714,  0.04727714, -0.00969204]
    ]
)


In [6]:
# Register with device specs
device = RydbergDevice(rydberg_level = 70)

reg = Register.from_coordinates(
    qubo_register_coords(Q),
    device_specs = device
)

# Analog circuit
layers = 2

block = chain(*[AnalogRX(f"t{i}") * AnalogRZ(f"s{i}") for i in range(layers)])

In [12]:
model = QuantumModel(
    QuantumCircuit(reg, block),
    backend="pyqtorch", diff_mode='gpsr')
initial_counts = model.sample({}, n_shots=1000)[0]
initial_counts

Counter({'0000': 164,
         '0001': 99,
         '0100': 92,
         '1000': 72,
         '0010': 71,
         '1100': 64,
         '0011': 54,
         '1001': 54,
         '0101': 51,
         '1010': 47,
         '1110': 47,
         '0110': 46,
         '1011': 40,
         '0111': 34,
         '1111': 34,
         '1101': 31})

In [8]:
def loss(param, *args):
    Q = args[0]
    param = torch.tensor(param)
    model.reset_vparams(param)
    C = model.sample({}, n_shots=1000)[0]
    return cost_fn(C, Q)

In [9]:
# Optimization loop.
for i in range(20):
    res = minimize(
        loss,
        args=Q,
        x0=np.random.uniform(1, 10, size=2 * layers),
        method="COBYLA",
        tol=1e-8,
        options={"maxiter": 20},
    )

# Sample and visualize the optimal solution.
model.reset_vparams(res.x)
optimal_count = model.sample({}, n_shots=1000)[0]

In [11]:
optimal_count

Counter({'0000': 833,
         '0010': 50,
         '1000': 49,
         '0001': 38,
         '0100': 28,
         '0101': 1,
         '1001': 1})